# Test model example - HAY_AIS_HILLOCK

In [ ]:
import json
import sys
import os
import time
import neuron
from pathlib import Path

import matplotlib.pyplot as plt
import neuroplotlib as nplt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import shutil

import numpy as np

import model
import evaluator
import plotting
import utils

# %matplotlib notebook
%matplotlib notebook

In [ ]:
probe_type = "planar"
model_name = "hay_ais_hillock" #'hallermann' # "hay" hay_ais
model_folder = (Path(".") / f"{model_name}_model").absolute()

In [ ]:
protocols_file = "../data/hay_ecode_probe_planar/efeatures/protocols_BPO.json"
features_file = "../data/hay_ecode_probe_planar/efeatures/features_BPO.json"

In [ ]:
ais_recordings = utils.get_ais_hillock_extra_recordings()
ais_recordings

In [ ]:
cell = model.create(model_name=model_name, release=True)

probe = model.define_electrode(probe_type=probe_type)
# probe=None

param_names = [param.name for param in cell.params.values() if not param.frozen]

fitness_protocols = evaluator.define_protocols(
    model_name,
    feature_set="soma",
    feature_file=features_file,
    protocols_file=protocols_file,
    electrode=probe,
    protocols_with_lfp=["IDrest_300"],
    extra_recordings=dict(IDrest_300=ais_recordings)  
)

fitness_calculator = ephys.objectivescalculators.ObjectivesCalculator([])

sim = ephys.simulators.LFPySimulator(cell, cvode_active=True, electrode=probe, mechs_folders=model_folder)

eva = ephys.evaluators.CellEvaluator(                    
        cell_model=cell,                                                       
        param_names=param_names,                                                    
        fitness_protocols=fitness_protocols,                                        
        fitness_calculator=fitness_calculator,                                      
        sim=sim)

In [ ]:
idrest_300 = fitness_protocols["IDrest_300"]
# print(eva)

In [ ]:
idrest_300.recordings

In [ ]:
t_start = time.time()
responses = eva.run_protocols(protocols=fitness_protocols.values(), param_values={})
print(f"Elapsed time: {time.time() - t_start}")

In [ ]:
responses["IDrest_250.soma.v"]

In [ ]:
plotting.plot_responses(responses)

## Validate AIS channel distributions

In [ ]:
cell.freeze({})
cell.instantiate(sim)

In [ ]:
nplt.plot_detailed_neuron(cell.LFPyCell, plane="xy", color_ais="r", color_hillock="g")

In [ ]:
ais = cell.icell.ais[0]

In [ ]:
mechs = ais.psection()["density_mechs"]

plt.figure()
# plt.plot(mechs["Na12Hu2009"]["gNa12bar"] / np.max(mechs["Na12Hu2009"]["gNa12bar"] ), label="gNa12bar_Na12Hu2009")
# plt.plot(mechs["Na16Hu2009"]["gNa16bar"] / np.max(mechs["Na16Hu2009"]["gNa16bar"]) , label="gNa16bar_Na16Hu2009")
# plt.plot(mechs["KdShu2007"]["gkbar"] / np.max(mechs["KdShu2007"]["gkbar"]) , label="gkbar_KdShu2007")

plt.plot(mechs["Na12Hu2009"]["gNa12bar"], label="gNa12bar_Na12Hu2009")
plt.plot(mechs["Na16Hu2009"]["gNa16bar"] , label="gNa16bar_Na16Hu2009")
plt.plot(mechs["KdShu2007"]["gkbar"] , label="gkbar_KdShu2007")
plt.legend()

In [ ]:
cell.destroy(sim)
cell.unfreeze({})

## Plot AP at soma and AIS locations

In [ ]:
target_protocol = "IDrest_300"
plt.figure()
for resp_name, response in responses.items():
    if target_protocol in resp_name and ".v" in resp_name and "begin" not in resp_name:
        plt.plot(response['time'], response['voltage'], label=resp_name)
plt.legend()